Extraction and quantification of market sentiment from textual data such as news headlines or social media posts related to stock tickers. It fetches recent text data using AlphaVantage Global News API, preprocesses and cleans the text for analysis, and then applies the VADER sentiment analyser to assign sentiment scores (positive, negaitve, neutral and compound) to each piece of text. 
Scores are then aggregated over chosen time intervals to create a time-aligned sentiment dataset that can be merged with market price data for further modeling and visualisation. 

In [21]:
# import libraries 
import requests 
import pandas as pd 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import feedparser
import re
import yfinance as yf


In [22]:
def fetch_alphavantage_news_df(api_url: str) -> pd.DataFrame: 
    """
    Fetches news data from Alphavantage Global News API URL and returns
    a cleaned pandas DataFrame with parsed dates.
    
    Parameters:
        api_url (str): Fully constructed API URL with key and parameters.
    
    Returns:
        pd.DataFrame: DataFrame with news articles.
    """
    response = requests.get(api_url)
    data = response.json()

    # Extract the "feed" list from the JSON response 
    news_list = data.get("feed", [])
    
    # Convert list of news dictionaries into a DataFrame
    df = pd.DataFrame(news_list)

    # Parse the published dates into datetime objects 
    df["time_published"] = pd.to_datetime(df["time_published"], errors = "coerce")

    return df

In [23]:
# Dynamically create the url in order to specify the ticker, date, and apikey
ALPHA_API = "F54RUITJVPIPTGZF"

def build_alphavantage_news_url(ticker, date):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date={date}&tickers={ticker}&apikey={ALPHA_API}"
    return url

In [24]:
news_url = build_alphavantage_news_url("AAPL", "10-08-2025")

In [25]:
news_df = fetch_alphavantage_news_df(news_url)

In [26]:
# Visualise the dataframe 
print(news_df.columns.tolist())
print(news_df.head())

['title', 'url', 'time_published', 'authors', 'summary', 'banner_image', 'source', 'category_within_source', 'source_domain', 'topics', 'overall_sentiment_score', 'overall_sentiment_label', 'ticker_sentiment']
                                               title  \
0  Google Slapped With $36 Million Fine Amid Anti...   
1  Billionaire Warren Buffett Sold 41% of Berkshi...   
2  Apple's Vision Pro Struggles To Gain Traction ...   
3  3 No-Brainer Warren Buffett Stocks to Buy Righ...   
4  Peter Lynch: 'Stock Market Has Been The Best P...   

                                                 url      time_published  \
0  https://www.benzinga.com/news/legal/25/08/4717... 2025-08-18 07:50:19   
1  https://www.fool.com/investing/2025/08/18/bill... 2025-08-18 07:06:00   
2  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-17 19:51:16   
3  https://www.fool.com/investing/2025/08/17/3-no... 2025-08-17 18:05:00   
4  https://www.benzinga.com/markets/guidance/25/0... 2025-08-17 17:46:27 

In [27]:
# Apply VADER sentiment analysis on the news headlines using the "title" and "summary" columns

sia = SentimentIntensityAnalyzer()
def get_sentiment_scores(text): 
    if isinstance(text, str):
        return sia.polarity_scores(text)
    else:
        return {'neg': None, 'neu': None, 'pos': None, 'compound': None}


In [28]:
# Create sentiment score columns 
news_df[["neg", "neu", "pos", "compound"]] = news_df["title"].apply(get_sentiment_scores).apply(pd.Series)

In [29]:
news_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,neg,neu,pos,compound
0,Google Slapped With $36 Million Fine Amid Anti...,https://www.benzinga.com/news/legal/25/08/4717...,2025-08-18 07:50:19,[Namrata Sen],Alphabet's Google GOOG GOOGL has agreed to pay...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.069724,Neutral,"[{'ticker': 'TTRAF', 'relevance_score': '0.206...",0.000,0.917,0.083,0.2023
1,Billionaire Warren Buffett Sold 41% of Berkshi...,https://www.fool.com/investing/2025/08/18/bill...,2025-08-18 07:06:00,[Sean Williams],The Oracle of Omaha has sent more than 427 mil...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.276350,Somewhat-Bullish,"[{'ticker': 'DMPZF', 'relevance_score': '0.141...",0.000,1.000,0.000,0.0000
2,Apple's Vision Pro Struggles To Gain Traction ...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-17 19:51:16,[Bibhu Pattnaik],Apple's Vision Pro headset struggles to captiv...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.124842,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.2911...",0.109,0.655,0.236,0.4404
3,3 No-Brainer Warren Buffett Stocks to Buy Righ...,https://www.fool.com/investing/2025/08/17/3-no...,2025-08-17 18:05:00,[Jennifer Saibil],Each of these companies has robust long-term o...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.309107,Somewhat-Bullish,"[{'ticker': 'DMPZF', 'relevance_score': '0.045...",0.000,1.000,0.000,0.0000
4,Peter Lynch: 'Stock Market Has Been The Best P...,https://www.benzinga.com/markets/guidance/25/0...,2025-08-17 17:46:27,[Bibhu Pattnaik],"Lynch emphasizes the virtue of patience, highl...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.305453,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1889...",0.000,0.931,0.069,0.3818
5,"Coca-Cola, Amazon, Google, And Nvidia Have Use...",https://www.benzinga.com/news/topics/25/08/471...,2025-08-17 16:31:46,[Paula Tudoran],"In the same Palo Alto, California, building wh...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'IPO', 'relevance_score': '0.158519...",0.144115,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.1585...",0.000,0.946,0.054,0.1779
6,Warren Buffett Is Selling Apple and Bank of Am...,https://www.fool.com/investing/2025/08/17/warr...,2025-08-17 15:23:00,[Bram Berkowitz],Berkshire Hathaway recently filed its second-q...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.081731,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.2409...",0.000,1.000,0.000,0.0000
7,Apple Accidentally Exposes Top-Secret Hardware...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-17 13:31:10,[Bibhu Pattnaik],Apple's code slip-up reveals a sneak peek into...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.106031,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.9237...",0.231,0.769,0.000,-0.4404
8,Consumer Tech News ( August 11-August 15 ) : ...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-17 13:21:21,[Nabaparna Bhattacharya],"U.S. sentiment dips on inflation worries, whil...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.093983,Neutral,"[{'ticker': 'BABA', 'relevance_score': '0.0611...",0.000,1.000,0.000,0.0000
9,Meet the Marvelous V

In [30]:
news_df.isnull().sum()

title                      0
url                        0
time_published             0
authors                    0
summary                    0
banner_image               0
source                     0
category_within_source     0
source_domain              0
topics                     0
overall_sentiment_score    0
overall_sentiment_label    0
ticker_sentiment           0
neg                        0
neu                        0
pos                        0
compound                   0
dtype: int64

In [31]:
### Exploration of Sentiment Data 

## Structural inspection 
# Check the first few rows 
print(news_df.head())

# Check the column names 
print("\nColumn names: ")
print(news_df.columns.tolist())

# Check data types and missing values 
print("\nDataFrame info:")
print(news_df.info())

# Check for missing values 
print("\nMissing values per column:")
print(news_df.isnull().sum())

# Quick statistics for numerical columns: 
print("\nSummary statistics:")
print(news_df.describe())

                                               title  \
0  Google Slapped With $36 Million Fine Amid Anti...   
1  Billionaire Warren Buffett Sold 41% of Berkshi...   
2  Apple's Vision Pro Struggles To Gain Traction ...   
3  3 No-Brainer Warren Buffett Stocks to Buy Righ...   
4  Peter Lynch: 'Stock Market Has Been The Best P...   

                                                 url      time_published  \
0  https://www.benzinga.com/news/legal/25/08/4717... 2025-08-18 07:50:19   
1  https://www.fool.com/investing/2025/08/18/bill... 2025-08-18 07:06:00   
2  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-17 19:51:16   
3  https://www.fool.com/investing/2025/08/17/3-no... 2025-08-17 18:05:00   
4  https://www.benzinga.com/markets/guidance/25/0... 2025-08-17 17:46:27   

             authors                                            summary  \
0      [Namrata Sen]  Alphabet's Google GOOG GOOGL has agreed to pay...   
1    [Sean Williams]  The Oracle of Omaha has sent more 

All the columns have the expected names and types 
There are 0 missing values 
The time_published is a datetime object which is to be expected

In [32]:
### Extract relevant ticker-level information from the ticker_sentiment column 

# Extract "ticker" and "relevance_score" into separate columns 
news_df_exploded = news_df.explode("ticker_sentiment").reset_index(drop = True)
ticker_sentiment_expanded = pd.json_normalize(news_df_exploded["ticker_sentiment"])
news_df_final = pd.concat([news_df_exploded.drop(columns = ["ticker_sentiment"]), ticker_sentiment_expanded], axis = 1)
news_df_final.drop(columns = "category_within_source", inplace = True)
news_df_final

,title,url,time_published,authors,summary,banner_image,source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,neg,neu,pos,compound,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label
0,Google Slapped With $36 Million Fine Amid Anti...,https://www.benzinga.com/news/legal/25/08/4717...,2025-08-18 07:50:19,[Namrata Sen],Alphabet's Google GOOG GOOGL has agreed to pay...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.069724,Neutral,0.000,0.917,0.083,0.2023,TTRAF,0.20698,-0.008267,Neutral
1,Google Slapped With $36 Million Fine Amid Anti...,https://www.benzinga.com/news/legal/25/08/4717...,2025-08-18 07:50:19,[Namrata Sen],Alphabet's Google GOOG GOOGL has agreed to pay...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.069724,Neutral,0.000,0.917,0.083,0.2023,GOOG,0.810464,-0.008203,Neutral
2,Google Slapped With $36 Million Fine Amid Anti...,https://www.benzinga.com/news/legal/25/08/4717...,2025-08-18 07:50:19,[Namrata Sen],Alphabet's Google GOOG GOOGL has agreed to pay...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.069724,Neutral,0.000,0.917,0.083,0.2023,AAPL,0.138863,-0.247383,Somewhat-Bearish
3,Google Slapped With $36 Million Fine Amid Anti...,https://www.benzinga.com/news/legal/25/08/4717...,2025-08-18 07:50:19,[Namrata Sen],Alphabet's Google GOOG GOOGL has agreed to pay...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.069724,Neutral,0.000,0.917,0.083,0.2023,SNGNF,0.069697,0.0582,Neutral
4,Billionaire Warren Buffett Sold 41% of Berkshi...,https://www.fool.com/investing/2025/08/18/bill...,2025-08-18 07:06:00,[Sean Williams],The Oracle of Omaha has sent more than 427 mil...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.276350,Somewhat-Bullish,0.000,1.000,0.000,0.0000,DMPZF,0.141726,0.232348,Somewhat-Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,1 Monster Growth Stock to Buy Now -- Its Techn...,https://www.fool.com/investing/2025/08/14/1-gr...,2025-08-14 07:55:00,[Trevor Jennewine],Meta Platforms not only has a strong presence ...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.347646,Somewhat-Bullish,0.000,0.874,0.126,0.3818,META,0.479683,0.567123,Bullish
304,1 Monster Growth Stock to Buy Now -- Its Techn...,https://www.fool.com/investing/2025/08/14/1-gr...,2025-08-14 07:55:00,[Trevor Jennewine],Meta Platforms not only has a strong presence ...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.347646,Somewhat-Bullish,0.000,0.874,0.126,0.3818,AAPL,0.085324,0.078504,Neutral
305,1 Monster Growth Stock to Buy Now -- Its Techn...,https://www.fool.com/investing/2025/08/14/1-gr...,2025-08-14 07:55:00,[Trevor Jennewine],Meta Platforms not only has a strong presence ...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.347646,Somewhat-Bullish,0.000,0.874,0.126,0.3818,MS,0.042723,0.170171,Somewhat-Bullish
306,Elon Musk's Bid To Dismiss OpenAI's Harassment...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-14 03:26:53,[Ananya Gairola],Elon Musk will have to defend against OpenAI's...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",-0.171481,Somewhat-Bearish,0.408,0.592,0.000,-0.9022,AAPL,0.309539,-0.183671,Somewhat-Bearish


In [33]:
### Aggregate sentiment data at the level appropriate such that it matches the market data- daily per ticker 

# Sort the dataframe by ticker 
news_df_sorted = news_df_final.sort_values(by = "ticker")

# Convert time_published to date only 
news_df_sorted["time_published"] = pd.to_datetime(news_df_sorted["time_published"], errors = "coerce")
news_df_sorted["date"] = news_df_sorted["time_published"].dt.day

# Sort the values by ticker and date 
news_df_sorted.sort_values(by = ["ticker", "date"], inplace = True)

## Combine the multiple sentiment scores into a single summary value per group 

# New column with weighted sentiment per article 
news_df_sorted["ticker_sentiment_score"] = pd.to_numeric(news_df_sorted["ticker_sentiment_score"], errors = "coerce")
news_df_sorted["relevance_score"] = pd.to_numeric(news_df_sorted["relevance_score"], errors = "coerce")
news_df_sorted["weighted_sentiment_score"] = news_df_sorted["ticker_sentiment_score"] * news_df_sorted["relevance_score"]

# Group by ticket and date and then aggregate to sum the weighted sentiments, relevance scores and compute the weighted average sentiment by dividng these sums for each news article 
grouped = news_df_sorted.groupby(["ticker", "date"]).agg(
    total_weighted_sentiment = ("weighted_sentiment_score", "sum"),
    total_relevance = ("relevance_score", "sum")
).reset_index()  # optional, to turn MultiIndex into columns

grouped["weighted_avg_sentiment"] = grouped["total_weighted_sentiment"] / grouped["total_relevance"]

grouped = grouped.rename(columns = {"ticker": "Ticker", "date":"Date"})
grouped

,Ticker,Date,total_weighted_sentiment,total_relevance,weighted_avg_sentiment
0,0IJU,14,0.000000,0.089797,0.000000
1,AAP,17,-0.001472,0.061159,-0.024074
2,AAPL,14,0.359404,4.660220,0.077122
3,AAPL,15,1.008961,5.484604,0.183962
4,AAPL,16,0.695160,2.733720,0.254291
...,...,...,...,...,...
172,WRD,17,0.000980,0.061159,0.016019
173,XIACY,15,0.012475,0.058188,0.214390
174,XOM,14,0.013127,0.125134,0.104903
175,XPEV,15,0.012475,0.058188,0.214390


In [34]:
### Coordinate the sentiment data and the market data 

# Determine date range from sentiment data to define the market data window
start_date = (news_df_final["time_published"]).dt.date.min()
end_date = (news_df_final["time_published"]).dt.date.max()

# Extract unique tickers from news_df_final["ticker"]
unique_tickers = news_df_final["ticker"].unique()

print(start_date, end_date)

2025-08-14 2025-08-18


In [35]:
unique_tickers

array(['TTRAF', 'GOOG', 'AAPL', 'SNGNF', 'DMPZF', 'BAC', 'BRK-A', 'DPZ',
       'CRYPTO:INCNT', 'KO', 'AXP', 'AMZN', 'MSFT', 'META', 'NVDA',
       'EBAY', 'ADBE', 'UBER', 'UNH', 'BABA', 'XPEV', 'AMAT', 'DE',
       'CSCO', 'BYDDF', 'MS', 'WRD', 'ORCL', 'GS', 'GRAB', 'OPEN', 'AAP',
       'TSLA', 'NIO', 'VWAGY', 'AMC', 'TPR', 'HOOD', 'BIRK', 'COST', 'MA',
       'MDNDF', 'BA', 'SSNLF', 'WBD', 'MSI', 'GLW', 'ON', 'MSTR', 'PYPL',
       'FOREX:USD', 'CRYPTO:BTC', 'CRYPTO:ETH', 'HD', 'UAL', 'BEKE',
       'NRG', 'AVGO', 'BSQKZ', 'BIDU', 'MHK', 'PDD', 'AMD', 'LRCX', 'WHR',
       'MU', 'GT', 'INTC', 'VST', 'CZR', 'TSM', 'CVX', 'KHC', 'MCO',
       'MSBHF', 'T', 'DASH', 'TRI', 'SHOP', 'LEN', 'BYDDY', 'XIACY',
       'DVA', 'TXN', 'GFS', 'TMUS', 'HEI', '0IJU', 'STZ', 'LAMR', 'POOL',
       'ALLE', 'NUE', 'ARM', 'QCOM', 'MASI', 'AGPPF', 'AWK', 'NEM', 'RL',
       'CASY', 'ADP', 'CRYPTO:NEM', 'UI', 'NOK', 'SMCI', 'MRVL', 'IVZ',
       'NFLX', 'C', 'MORN', 'SCHW', 'XOM', 'PLTR'], dtype=object)

In [36]:
# Check Yahoo Finance download results

data = yf.download(unique_tickers.tolist(), start=start_date, end=end_date, group_by='ticker')
print("Downloaded data shape:", data.shape)
print("Downloaded data columns:", data.columns[:10])  # first 10 for preview
print("First few rows:\n", data.head())

/var/folders/34/mh55f40j11qcbq5hh0hfw_3w0000gn/T/ipykernel_26711/1111849683.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(unique_tickers.tolist(), start=start_date, end=end_date, group_by='ticker')
[**************        30%                       ]  34 of 114 completedHTTP Error 404: 
HTTP Error 404: 
[*********************100%***********************]  114 of 114 completed

6 Failed downloads:
['CRYPTO:BTC', 'FOREX:USD', 'CRYPTO:ETH', '0IJU', 'CRYPTO:NEM', 'CRYPTO:INCNT']: YFTzMissingError('possibly delisted; no timezone found')


Downloaded data shape: (2, 576)
Downloaded data columns: MultiIndex([(  'ON',   'Open'),
            (  'ON',   'High'),
            (  'ON',    'Low'),
            (  'ON',  'Close'),
            (  'ON', 'Volume'),
            ('TSLA',   'Open'),
            ('TSLA',   'High'),
            ('TSLA',    'Low'),
            ('TSLA',  'Close'),
            ('TSLA', 'Volume')],
           names=['Ticker', 'Price'])
First few rows:
 Ticker             ON                                                  TSLA  \
Price            Open       High        Low      Close   Volume        Open   
Date                                                                          
2025-08-14  50.799999  52.029999  49.709999  51.619999  7110100  335.760010   
2025-08-15  52.110001  52.110001  50.840000  51.090000  7572300  337.660004   

Ticker                                                    ...        LAMR  \
Price             High         Low       Close    Volume  ...        Open   
Date             

In [37]:
data

Ticker             ON                                                  TSLA  \
Price            Open       High        Low      Close   Volume        Open   
Date                                                                          
2025-08-14  50.799999  52.029999  49.709999  51.619999  7110100  335.760010   
2025-08-15  52.110001  52.110001  50.840000  51.090000  7572300  337.660004   

Ticker                                                    ...        LAMR  \
Price             High         Low       Close    Volume  ...        Open   
Date                                                      ...               
2025-08-14  340.470001  330.399994  335.579987  75000700  ...  120.910004   
2025-08-15  339.299988  327.019989  330.559998  74157400  ...  124.000000   

Ticker                                                          UNH  \
Price             High         Low       Close   Volume        Open   
Date                                                                  
2025-08-14  122.000000  120.129997  121.220001   710800  272.100006   
2025-08-15  125.790001  122.500000  123.269997  1583300  301.709991   

Ticker                                                   
Price             High         Low      Close    Volume  
Date                                                     
2025-08-14  273.850006  267.000000  271.48999  25011100  
2025-08-15  310.299988  294.709991  304.01001  67987200  

[2 rows x 576 columns]

In [38]:
# Download data from yfinance dataset 

def fetch_and_process_daily_data(tickers, start_date, end_date):
    if isinstance(tickers, str):
        tickers = [tickers]

    df = yf.download(tickers.tolist(), start=start_date, end=end_date, interval="1d", group_by=None)

    if isinstance(df.columns, pd.MultiIndex):
        df = df.stack(level=0, future_stack=True).reset_index()
        df = df.rename(columns={'level_1': 'Ticker'})
    else:
        df = df.reset_index()
        df['Ticker'] = tickers[0]

    df["log_return"] = df.groupby("Ticker")["Close"].transform(lambda x: np.log(x / x.shift(1)))
    print(df[["Ticker", "Date", "Close", "log_return"]].head(10))
    print(df.groupby("Ticker").size())
    df = df.dropna(subset=["log_return"])

    return df


In [39]:
final_df = fetch_and_process_daily_data(unique_tickers, "2025-03-03", "2025-08-12")

/var/folders/34/mh55f40j11qcbq5hh0hfw_3w0000gn/T/ipykernel_26711/2074349093.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers.tolist(), start=start_date, end=end_date, interval="1d", group_by=None)
[*********************100%***********************]  114 of 114 completed

6 Failed downloads:
['FOREX:USD', 'CRYPTO:BTC', 'CRYPTO:ETH', '0IJU', 'CRYPTO:NEM', 'CRYPTO:INCNT']: YFTzMissingError('possibly delisted; no timezone found')


Price Ticker       Date       Close  log_return
0       SMCI 2025-03-03   36.070000         NaN
1       AMZN 2025-03-03  205.020004         NaN
2       NVDA 2025-03-03  114.041588         NaN
3        TRI 2025-03-03  177.956116         NaN
4       ADBE 2025-03-03  440.720001         NaN
5       TSLA 2025-03-03  284.649994         NaN
6        DPZ 2025-03-03  479.997101         NaN
7        AAP 2025-03-03   34.987442         NaN
8         ON 2025-03-03   44.910000         NaN
9        WBD 2025-03-03   11.020000         NaN
Ticker
0IJU     112
AAP      112
AAPL     112
ADBE     112
ADP      112
        ... 
WHR      112
WRD      112
XIACY    112
XOM      112
XPEV     112
Length: 114, dtype: int64


/opt/miniconda3/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [40]:
### Process sentiment data 

# Group by date & ticker to get daily average sentiment using weighted sentiment score
sentiment_daily = (
    grouped
    .groupby(["Date", "Ticker"])["weighted_avg_sentiment"]
    .mean()
    .reset_index()
)

# Make sure both are Date columns are datetime 
final_df["Date"] = pd.to_datetime(final_df["Date"]).dt.normalize()
sentiment_daily["Date"] = pd.to_datetime(sentiment_daily["Date"]).dt.normalize()

### Process market data
final_df["Date"] = pd.to_datetime(final_df["Date"])

### Merge market and sentiment data 
merged_df = pd.merge(final_df, sentiment_daily, on=["Date", "Ticker"], how="left")

In [41]:
merged_df

,Date,Ticker,Open,High,Low,Close,Volume,Adj Close,log_return,weighted_avg_sentiment
0,2025-03-04,SMCI,35.000000,40.610001,34.509998,39.139999,109691900.0,NaN,0.081683,NaN
1,2025-03-04,AMZN,200.110001,206.800003,197.429993,203.800003,60853100.0,NaN,-0.005968,NaN
2,2025-03-04,NVDA,110.632141,119.290739,110.092227,115.971275,398163300.0,NaN,0.016779,NaN
3,2025-03-04,TRI,178.145509,178.643902,174.716564,175.254837,322100.0,NaN,-0.015296,NaN
4,2025-03-04,ADBE,439.250000,448.820007,434.700012,446.029999,2942200.0,NaN,0.011976,NaN
...,...,...,...,...,...,...,...,...,...,...
11981,2025-08-11,LAMR,114.779999,116.309998,113.660004,114.980003,829700.0,NaN,-0.004512,NaN
11982,2025-08-11,HEI,313.720001,315.589996,306.029999,312.470001,257800.0,NaN,-0.002940,NaN
11983,2025-08-11,PDD,114.430000,114.610001,112.050003,113.040001,4445800.0,NaN,-0.010735,NaN
11984,2025-08-11,AMAT,187.149994,189.720001,183.779999,184.380005,7816000.0,NaN,-0.002654,NaN
